In [ ]:
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, RFECV, SequentialFeatureSelector
import random
from scipy import stats

# from collections import defaultdict
# from itertools import islice
# from scipy.stats import permutation_test
# import statsmodels.api as sm
# from statsmodels.genmod.generalized_linear_model import GLM

In [ ]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): 
    
    word = re.sub(' ', '_', word) 
    
    word = re.sub(r'\'', '', word) 
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    word = word.lower()
    
    return word

In [ ]:
df_gen = pd.read_csv("full_gen.csv")

In [ ]:
#READ IN Updated CLINICAL DATA FOR LATER USE (CONVERTED TO .csv IN GOOGLE SHEETS)
df_clin_updated = pd.read_csv("Homebase_new_updated.csv", header = 1)

In [ ]:
#RENAMING COLUMNS
df_clin_updated = df_clin_updated.rename(mapper = lower_no_space, axis = 1) 
df_clin_updated.rename(columns={'subject_sample_id':'sample_id'}, inplace=True)

In [ ]:
#Compute the age at initial diagnosis from date of birth and date_of_initial_diagnosis
df_clin_updated['date_of_birth'] = pd.to_datetime(df_clin_updated['date_of_birth'])
df_clin_updated['date_of_initial_diagnosis'] = pd.to_datetime(df_clin_updated['date_of_initial_diagnosis'])
df_clin_updated["age_at_initial_diagnosis"] = (pd.DatetimeIndex(df_clin_updated['date_of_initial_diagnosis']).year 
                        - pd.DatetimeIndex(df_clin_updated['date_of_birth']).year)

In [ ]:
#Due to the abnormal in date of birth from the Stanford data, 
#Remove the age at initial diagonosis for data from Stanford & the one that has negative age 
df_clin_updated["age_at_initial_diagnosis"] = np.where(df_clin_updated['data_access_group'] == 'Stanford', np.nan, df_clin_updated["age_at_initial_diagnosis"])
df_clin_updated["age_at_initial_diagnosis"] = np.where(df_clin_updated["age_at_initial_diagnosis"] < 0, np.nan, df_clin_updated["age_at_initial_diagnosis"])


In [ ]:
#Change the data type: date_of_birth, n, m 
df_clin_updated = df_clin_updated.astype({'t':'object', 'b':'object'})


In [ ]:
#TONS OF DATA, PULL WHAT WE WANT
df_clin_updated_lean = df_clin_updated.drop(columns = [x for x in df_clin_updated.columns if x not in ['gender', 'race', \
                                       'country_of_residence', 'sample_id', 'ethnicity',\
                                        'age_at_initial_diagnosis', 't', 'n', 'm', 'b',\
                                        'predominant_lesion_type_at_diagnosis','lymph_node_biopsy_performed',\
                                        'family_history_of_leukemia_lymphoma', \
                                        'has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical',\
                                        'cd4+:cd8+_ratio', 'total_lymphocyte_count', 'absolute_cd4+_count_per_ul',\
                                        '%cd4+cd26-', '%cd4+cd7-', 'tcr_clonality', 'tumor_cell_cd30+',\
                                        'large_cell_transformation', 'ldh_u_l', 'wbc_103_μl', 'rbc_106_μl',\
                                        'hematocrit_%', 'mcv_fl', 'mchc_g_dl', 'rdw_%', 'platelet_count_103_μl',\
                                        'segmented_neutrophil,_absolute_103_μl', 'lymphocyte,_absolute_103_μl',\
                                        'monocytes,_absolute_103_μl', 'eosinophils,_absolute_103_μl',\
                                        'basophils,_absolute_103_μl', 'segmented_neutrophils_%', 'lymphocytes_%',\
                                        'monocytes_%', 'eosinophils_%', 'basophils_%']])

In [ ]:
# TURN YES/NO & POSITIVE/NEGATIVE TO DUMMIES
df_clin_updated_lean['lymph_node_biopsy_performed'] = \
df_clin_updated_lean['lymph_node_biopsy_performed'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['family_history_of_leukemia_lymphoma'] = \
df_clin_updated_lean['family_history_of_leukemia_lymphoma'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['tumor_cell_cd30+'] = \
df_clin_updated_lean['tumor_cell_cd30+'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['large_cell_transformation'] = \
df_clin_updated_lean['large_cell_transformation'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['tcr_clonality'] = \
df_clin_updated_lean['tcr_clonality'].apply(lambda x: 1 if x == 'Positive' else 0)

df_clin_updated_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'] = \
df_clin_updated_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
# Read in the Preprocessed Genetic Data
df_lean = pd.read_csv ('stats_by_sample.csv')

In [ ]:
df_lean.head()

In [ ]:
#TRANSFORM SAMPLE ID TO JOIN TO CLINICAL DATA
df_lean['sample_id'] = df_lean['sample_id'].apply(lambda x: re.sub('_', '-', x[:5]) if 'WES' in x else\
                                                  (x[:-10] if 'CTCL' in x else \
                                                  (x[:-13] if 'almeida' in x else\
                                                  ((x[-2:]+x[:-2])[:-15] if 'ungewickell' in x else\
                                                  ('-'.join([ele.lstrip('0').lower() for ele in x[:-10].split('-')]) if 'SPZ' in x else x)))))

In [ ]:
#MERGE tbe updated CLINICAL, GENETIC DATA
df_all_updated = pd.merge(df_lean, df_clin_updated_lean, on='sample_id', how='left')

In [ ]:
#IMPUTATION; "UNKNOWN" FOR CATEGORICAL, MEAN FILL-IN FOR CONTINUOUS
for col in df_clin_updated_lean.columns:
    if col in ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis', 't', 
              'n', 'm', 'b']:
        df_all_updated[col] = df_all_updated[col].fillna('unknown')
    elif col != 'sample_id':
        df_all_updated[col] = df_all_updated[col].fillna(np.mean(df_all_updated[col]))

In [ ]:
#GET DUMMIES FOR CATEGORICALS
df_all_updated = pd.get_dummies(df_all_updated, columns = ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis', 
                                                          't', 'n', 'm', 'b'])


In [ ]:
#DEFINE STANDARDSCALER FOR LATER USE
std_scl = StandardScaler()

In [ ]:
# Define (Scaled/Normalized) Features and Labels
X_new = df_all_updated.drop(columns = [x for x in df_all_updated.columns if x == 'outcome' or x == 'sample_id'])
X_new_scaled = std_scl.fit_transform(X_new)
X_new_norm = normalize(X_new)

y_new = df_all_updated.drop(columns = [x for x in df_all_updated.columns if x != 'outcome'])

In [ ]:
df_all_updated['outcome'].value_counts(normalize = True)

In [ ]:
# DEFINE CHROMOSOMES OF INTEREST
top_feats = pd.read_csv("ada_fi.csv").head(25).drop(columns = ['Unnamed: 0'])
top_feats
chroms_to_use = top_feats[top_feats['feature_names'].str.contains("chromosome")]['feature_names']
chroms_to_use = [re.sub('chromosome_', '', \
                        re.sub('_mutations', '', \
                               re.sub('_rawscore', '', \
                                      re.sub('_non_neg_rawscore', '', \
                                             re.sub('med_', '', x))))) \
                 for x in chroms_to_use]


chroms_to_use_2 = top_feats[(top_feats['feature_names'].str.contains("section")) &\
                           ~(top_feats['feature_names'].str.contains("chromosome"))]['feature_names']
chroms_to_use_2 = [re.sub('_mutations', '', \
                          re.sub('_rawscore', '', \
                                 re.sub('_non_neg_rawscore', '', \
                                        x.split("chrom_", 1)[1])))\
                  for x in chroms_to_use_2]


chroms_to_use += chroms_to_use_2
chroms_to_use = list(set(chroms_to_use))
chroms_to_use

In [ ]:
df_use = df_all_updated.copy()
cols_to_use = ['outcome']

for gene in set(df_gen[df_gen['chrom'].isin(chroms_to_use)]['gene_symbol']):
    if ('gene_' + gene + '_non_neg_rawscore') in df_lean.columns:
            cols_to_use.append('gene_' + gene + '_non_neg_rawscore')
df_use = df_use.drop(columns = [x for x in df_use.columns if x not in cols_to_use])

In [ ]:
log = lm.LogisticRegression()
rf = ens.RandomForestClassifier()
rdg = lm.RidgeClassifier(alpha = 1)
ada = ens.AdaBoostClassifier()


NEXT FEW CELLS ARE ABOUT GETTING P-VALS FROM SKLEARN;
METHODOLOGY LEARNED HERE:
https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

In [ ]:
X = df_use.drop(columns = [x for x in df_use.columns if x == 'sample_id' or x == 'outcome'])
y = df_use['outcome']

In [ ]:
class perm_test():
    def linear(self, model_type, ex, why):
        
        model = model_type.fit(ex, why.values.ravel())
        
        # DEFINE FULL MODEL PARAMS (INCLUDE B_0) AND PREDICTIONS YOU WOULD GET
        params = np.append(model.intercept_, model.coef_)
        predictions = model.predict(ex)
        
        # MAKE NEW DF TO STORE KEY VALUES, DEFINE MSE OF PREDS FROM PREV CELL
        newX = pd.DataFrame({"Constant":np.ones(len(ex))}).join(pd.DataFrame(ex))
        
        # DEFINE VAR, SD, T SCORE
        # DIAGONAL OF COVARIANCE MATRIX SHOULD EQUAL AN ARRAY OF ALL COEF VAR
        var_b = newX.cov().to_numpy().diagonal().copy()
        var_b += .0000000001 #ADDING VERY SMALL NUMBER TO DEAL WITH 0s
        sd_b = np.sqrt(var_b)
        ts_b = params/ sd_b #T STATISTIC = (ESTIMATED COEF - HYPOTHESIZED COEF (0))/COEF SAMPLE STANDARD ERROR
        
        p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX) - 1))) for i in ts_b]
        p_values = np.round(p_values,3)
        
        genes_comp = {}

        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))
            genes_comp[gene] = {}
            genes_comp[gene]['p_val'] = p_values[i + 1]
            
        random_best_p_values = []

        for i in range(10000):
            print(i)            
            temp_df = df_use.copy() #DON'T ALTER ORIG DF

            # GET CORRECTLY SIZED GROUPS, RANDOMLY ASSIGNED, SAME SPLITS AS IN ORIG
            ss_sized_group = temp_df.sample(n = len(set(df_gen[df_gen['outcome'] == 1]['sample_id'])))
            mf_sized_group = temp_df[~temp_df.index.isin(ss_sized_group.index.values)]

        #     RESET OUTCOME VALS BASED ON GROUP (NOW OUTCOME IS RANDOM)
            ss_sized_group['outcome'].values[:] = 1
            mf_sized_group['outcome'].values[:] = 0

        #     RECOMBINE INTO ONE DF
            new_df = pd.concat([ss_sized_group, mf_sized_group])
    
                # DEFINE X, Y, FITTED MODEL
            X = new_df.drop(columns = [x for x in new_df.columns if x == 'sample_id' or x == 'outcome'])
            y = new_df['outcome']
            
            model = model_type.fit(ex, why)
            
            params = np.append(model.intercept_, model.coef_)
            predictions = model.predict(ex)
            
            newX = pd.DataFrame({"Constant":np.ones(len(ex))}).join(pd.DataFrame(ex))
            
            var_b = newX.cov().to_numpy().diagonal().copy()
            var_b += .000000001
            sd_b = np.sqrt(var_b)
            ts_b = params/ sd_b
    
            p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX) - 1))) for i in ts_b]
            p_values = np.round(p_values,3)
            p_values = p_values[1:] #FIRST ONE WILL BE FOR CONSTANT (SHOULD EQUAL 0)
    
            baseline = np.min(p_values)
            random_best_p_values.append(baseline)
            
        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))

            genes_comp[gene]['p_val'] = len([x for x in random_best_p_values if x < genes_comp[gene]['p_val']])/10000
    
        genes_comp_ordered = {k: v for k, v in sorted(genes_comp.items(), key=lambda item: item[1]['p_val'], reverse = False)}
        return genes_comp_ordered
    

            
    def tree_based(self, model_type, ex, why):
        
        model = model_type.fit(ex, why.values.ravel())
        
        genes_comp = {}

        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))
            genes_comp[gene] = {}
            genes_comp[gene]['fi'] = model.feature_importances_[i]
            
        random_best_fi_vals = []

        for i in range(10000):
            print(i)            
            temp_df = df_use.copy() #DON'T ALTER ORIG DF

            ss_sized_group = temp_df.sample(n = len(set(df_gen[df_gen['outcome'] == 1]['sample_id'])))
            mf_sized_group = temp_df[~temp_df.index.isin(ss_sized_group.index.values)]

            ss_sized_group['outcome'].values[:] = 1
            mf_sized_group['outcome'].values[:] = 0

            new_df = pd.concat([ss_sized_group, mf_sized_group])

            X = new_df.drop(columns = [x for x in new_df.columns if x == 'sample_id' or x == 'outcome'])
            y = new_df['outcome']
            model = model_type.fit(ex, why)
            
            baseline = np.max(model.feature_importances_)
            random_best_fi_vals.append(baseline)
            
        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))

            genes_comp[gene]['p_val'] = len([x for x in random_best_fi_vals if x > genes_comp[gene]['fi']])/10000
    
        genes_comp_filtered = {}

        for (key, value) in genes_comp.items():
           # Check if key is even then add pair to new dictionary
           if value['fi'] != 0:
               genes_comp_filtered[key] = value
            
        genes_comp_ordered = {k: v for k, v in sorted(genes_comp_filtered.items(), key=lambda item: item[1]['p_val'], reverse = False)}
        return genes_comp_ordered


In [ ]:
print("LOGISTIC REGRESSION \n")
log_perm = perm_test()
log_perm_result = log_perm.linear(log, X, y)
print("RIDGE CLASSIFIER \n")
rdg_perm = perm_test()
rdg_perm_result = rdg_perm.linear(rdg, X, y)
print("ADABOOST \n")
ada_perm = perm_test()
ada_perm_result = ada_perm.tree_based(ada, X, y)
print("RANDOM FOREST \n")
rf_perm = perm_test()
rf_perm_result = rf_perm.tree_based(rf, X, y)

In [ ]:
ada_df = pd.DataFrame.from_dict(ada_perm_result)
ada_df.to_csv("ada_dict.csv")
rf_df = pd.DataFrame.from_dict(rf_perm_result)
rf_df.to_csv("rf_dict.csv")
rdg_df = pd.DataFrame.from_dict(rdg_perm_result)
rdg_df.to_csv("rdg_dict.csv")
log_df = pd.DataFrame.from_dict(log_perm_result)
log_df.to_csv("log_dict.csv")

In [ ]:
ada_perm_result
# ADABOOST - IQSEC1: FI = .1, P-VAL = .0
    

In [ ]:
rf_perm_result
# RANDOM FOREST - NONE

In [ ]:
log_perm_result
# LOGISTIC REGRESSION - KRTAP5-5: P-VAL = 0; TPSD1: P-VAL = 0

In [ ]:
rdg_perm_result
# RIDGE CLASSIFIER - KRTAP5-5: P-VAL = 0; TPSD1: P-VAL = 0